In [585]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [4]:
#initialization

In [925]:
X =  np.array([[-4,0],[-7,0],[-1,0],[0,1],[2,1],[4,0],[5,1]])
S = np.array([[1],[0.2],[3]])
M =  np.array([[-4,0],[0,0],[8,0]])
P = np.array([[1/3],[1/3],[1/3]])
S.shape

(3, 1)

In [908]:
X =  np.array([[-3],[-2.5],[-1],[0],[2],[4],[5]])
S = np.array([[1],[0.2],[3]])
M =  np.array([[-4],[0],[8]])
P = np.array([[1/3],[1/3],[1/3]])

In [920]:
X =  np.array([[-3],[-2.5],[-1],[0],[2],[4],[5]])
S = np.array([[1],[0.2],[3]])
M =  np.array([[-4],[0],[8]])
P = np.array([[1/3],[1/3],[1/3]])

In [912]:
a = calc_G3(X, M, S.reshape((M.shape[0],X.shape[1],X.shape[1]) ), P)
a

ValueError: cannot reshape array of size 3 into shape (3,2,2)

In [926]:
b = calc_G2(X, M, S, P)
b

array([[1.00000000e+00, 2.12417713e-17, 1.25837818e-11],
       [1.00000000e+00, 2.83283409e-51, 1.55296205e-15],
       [2.63537099e-02, 9.73645206e-01, 1.08410206e-06],
       [4.95496852e-04, 9.99488487e-01, 1.60164349e-05],
       [1.28646572e-05, 2.59498657e-02, 9.74037270e-01],
       [5.46784824e-13, 9.17129370e-16, 1.00000000e+00],
       [2.48239926e-17, 4.68569560e-27, 1.00000000e+00]])

In [927]:
c = calc_G(X, M,  np.sqrt(S) , P)
c

array([[1.00000000e+00, 9.49960891e-18, 2.17957495e-11],
       [1.00000000e+00, 1.26688192e-51, 2.68980917e-15],
       [5.70694724e-02, 9.42926461e-01, 4.06623892e-06],
       [1.50000042e-04, 9.99843983e-01, 6.01744506e-06],
       [9.93708362e-06, 6.62368700e-02, 9.33753193e-01],
       [3.15686365e-13, 2.36801785e-16, 1.00000000e+00],
       [2.00021110e-17, 1.24761973e-26, 1.00000000e+00]])

In [877]:
def calc_G(X, M, S, P):  # returns an (N x K) array
    G = []
    for Xn in X:
        G.append([stats.norm(Mk,Sk).pdf(Xn) for Mk,Sk in zip(M,S)] )
    G = P.T * np.array(G)[:,:,0]
    G = G / np.sum(G , axis=1).reshape(-1,1)
    return G


def calc_G1(X, M, S, P):  # returns an (N x K) array
    G = []
    for Xn in X:
        g = []
        for k in range(M.shape[0]):
            g.append( stats.norm(M[k],S[k]).pdf(Xn)  )
        G.append(g)
    G = P.T * np.array(G)[:,:,0]
    G = G / np.sum(G , axis=1).reshape(-1,1)
    return G

def calc_G2(X, M, S, P):  # returns an (N x K) array
    G = np.zeros((X.shape[0], M.shape[0]))
    for k in range(M.shape[0]):
        first_part = 1 / np.sqrt(2 * np.pi * S[k])
        second_part = np.exp(-(1 / (2 * S[k])) * (X - M[k, :]) ** 2)
        G[:, k] = np.prod(first_part * second_part, axis=1)
    G = P.T * G
    G = G / np.sum(G , axis=1).reshape(-1,1)
    return G

def calc_G3(X, M, S, P):  # returns an (N x K) array
    G = np.zeros((X.shape[0], M.shape[0]))
    for k in range(M.shape[0]):
        prior = P[k]
        likelihood = stats.multivariate_normal(M[k], S[k]).pdf(X)
        G[:, k] = prior * likelihood
        
    G = G / np.sum(G , axis=1).reshape(-1,1)
    return G


def calc_Nk(G):   # returns an (1 x K) array
    return np.sum(G , axis=0).reshape(-1,1)

def calc_Nk2(G):   # returns an (1 x K) array
    N = []
    for k in range(G.shape[1]):
        nk = 0
        for n in range(G.shape[0]):
            nk += G[n][k]
        N.append(nk)
    return np.array(N).reshape(-1,1)
    
def calc_M(X, G, Nk): # returns an (K x D) array
    return G.T.dot(X) / Nk

def calc_M2(X, G, Nk): # returns an (K x D) array
    M = []
    for k in range(G.shape[1]):
        row = 0
        for n in range(X.shape[0]):
            row += G[n][k] * X[n]
        M.append(row)
    
    return np.array(M)/Nk
            

def calc_S(X, G, Nk, M):
    S = []
    for k in range(M.shape[0]):
        row = 0
        for n in range(X.shape[0]):
            for d in range(X.shape[1]):
                row += G[n][k]*np.square(X[n][d]- M[k][d])
        S.append(row)
    return np.array(S).reshape(-1,1) /(X.shape[1] * np.sum(Nk) )

In [883]:
g = calc_G(X, M, S, P)
nk =calc_Nk(g)
m = calc_M(X, g, nk)
calc_S(X, g , nk , m)

array([[0.04232009],
       [0.12584779],
       [0.6398094 ]])

In [879]:
g = calc_G(X, M, S, P)
nk =calc_Nk2(g)
m = calc_M2(X, g, nk)
calc_S(X, g , nk , m)

array([[0.40204494],
       [0.09866386],
       [0.36698623]])

In [874]:
while True:

    
    break